In [1]:
#Import libraries
import sys
import cv2
import pathlib
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers
import keras
from glob import glob, iglob
from sklearn.metrics import confusion_matrix
import itertools
import pickle
from matplotlib.ticker import MaxNLocator
import operator
from decimal import Decimal
from sklearn.model_selection import train_test_split
import shutil

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Global Variable

In [ ]:
directory = "" 

# Extracting Faces & Resizing

In [3]:
def extract_face(img_list, min_size = (200,200)):
  for i in range(len(img_list)):
    img = img_list[i]
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor = 1.05, minNeighbors = 5, minSize= min_size ) #, minSize = (200,200)) 
  
    for (x, y, w, h) in faces:
      faces = img[y:y + h, x:x + w]
      faces = cv2.resize(faces, (224,224),interpolation = cv2.INTER_AREA)
    img_list[i] = faces

# Splitting video into images, extracting faces & resizing

In [4]:
def extract_images(pathIn): #, pathOut):
    to_return = []
    count = 0
    vidcap = cv2.VideoCapture(pathIn)
    success,image = vidcap.read()    #Grabs, decodes and returns the next video frame.
    success = True
    while success:
        vidcap.set(cv2.CAP_PROP_POS_MSEC,(count*500))    # added this line. set() = set a property in video capture #0.167 for 6 frames per sec -> 6 img/3 sec
        #CAP_PROP_POS_MSEC = Current position of the video file in milliseconds.
        success,image = vidcap.read()
        if success == True:    #If have read in a new frame
          #cv2.imwrite(os.path.join(pathOut + "/frame_%d_.jpg" % count), image)     # save frame as JPEG file
          to_return.append(image)
        count = count + 1

    return to_return #pathOut

In [5]:
def slice_video(video_name):
  #directory must have been set, will be a global variable
  sourceVideoDirectory = os.path.join(directory, video_name)
  #targetVideoDirectory = os.path.join(directory + "sliced", video_name)

  #Create the target video directory if does not exist. Reference: https://stackoverflow.com/questions/273192/how-can-i-safely-create-a-nested-directory-in-python
  # pathlib.Path(targetVideoDirectory).mkdir(parents=True, exist_ok=True)

  #Split the video into frames. 
  return extract_images(sourceVideoDirectory) #, targetVideoDirectory)

# Data duplication

In [6]:
def duplicate(img_list):
  n = len(img_list)
  if n % 6 != 0:
    duplicate_num = 6 - (n%6)

    for i in range(duplicate_num):
      img_list.append(img_list[-1])

  return len(img_list)

# Creating list of images in np.arr form

In [7]:
def transform_img(img_list):
  #img_list.sort(key = lambda x : int(x[1]))
  img_list = np.asarray(img_list)
  img_list = [np.expand_dims(img,0) for img in img_list]

  return img_list #return a list of tuple (np.arr of image, image code)

# Predicted label and time

In [21]:
def get_label_time(arr, threshold = 0.25):
  to_return = []
  time_str = "" 

  for i in range(len(arr)):
    prob = arr[i][0]
    idx = arr[i][1][0]

    if prob > threshold:
      hour = i*3 // 3600
      minute = i*3 // 60
      second = i*3 % 60

      if hour == 0 and minute == 0 and second == 0:
        pass
      else:
        if hour != 0:
          time_str += str(hour)
        
        # add minute
        if hour != 0 and len(str(minute)) == 1:
          time_str += ":0" + str(minute)
        
        else: #hour == 0 and len(minute)
          time_str += str(minute)

        # add second
        if len(str(second)) == 1:
          time_str += ":0" + str(second)

        else:
          time_str += ":" + str(second)

        to_return.append((label_code[idx], time_str))
      time_str = "" 

  return to_return

# Main function(?)

In [8]:
def main(file_name):
  # process file
  # 1 - Slice video
  extracted_img = slice_video(video_name)

  # 2 - Extract face only & resize image
  extract_face(extracted_img)

  # 3 - Do data duplication & tansfrom image
  duplicate(extracted_img)
  img_list = transform_img(extracted_img)

  # 5 - Do prediction
  predicted_list = []
  for i in range(0, len(img_list),6):
    input = img_list[i:i+6]
    yhat = trainedModel.predict(input) #Use verbose to get progress bar
    predicted_list.append((yhat.max(), np.argmax(yhat, axis=1)))

  return get_label_time(predicted_list)

In [24]:
#trainedModel = keras.models.load_model("/content/drive/Shareddrives/Final Year Project/CS 2/Deep Learning Models/finetunecheckpoint6epochs0.40acc.hdf5")
trainedModel = keras.models.load_model("/content/drive/Shareddrives/Final Year Project/CS 2/Deep Learning Models/savedManually34epochs0.5acc.h5")
#trainedModel = keras.models.load_model("/content/drive/Shareddrives/Final Year Project/CS 2/Deep Learning Models/0.6acc.h5")

In [10]:
label_code = {0: 'anger', 1: 'disgust', 2 : 'fear', 3 : 'happy', 4 : 'sad', 5 : 'surprise'}

In [11]:
directory = "/content/drive/Shareddrives/Final Year Project/CS 2/Deep Learning Quality Assurance/Test videos FER finalized/"
#main(video_name)

# Manual Testing

In [12]:
directory = "/content/drive/Shareddrives/Final Year Project/CS 2/Deep Learning Quality Assurance/Test videos FER finalized/"

In [13]:
file_name = ['1meterhappy.mp4', 'halfmetersad.mp4', 'halfmeterhappy.mp4', 'halfmeternoisybackgroundhappy.mp4', 'halfmeternoisybackgrounddisgust.mp4']

In [14]:
extracted_img = slice_video('grchappy.mp4')
extract_face(extracted_img)
duplicate(extracted_img)
img_list = transform_img(extracted_img)

In [15]:
predicted_list = []
for i in range(0, len(img_list),6):
  input = img_list[i:i+6]
  yhat = trainedModel.predict(input) # , verbose = 1) #Use verbose to get progress bar
  predicted_list.append((yhat.max(), np.argmax(yhat, axis=1)))
  print(yhat)

[[0.00813482 0.41792667 0.2910888  0.08951189 0.13876022 0.05457763]]
[[0.00813404 0.41798022 0.29108638 0.08950233 0.13872659 0.05457047]]
[[0.00813404 0.41798013 0.2910864  0.08950236 0.13872668 0.0545705 ]]
[[0.00892405 0.2610101  0.27910417 0.11207408 0.2662094  0.07267824]]
[[0.00813404 0.41798022 0.29108638 0.08950233 0.13872659 0.05457047]]
[[0.00813404 0.41798022 0.29108638 0.08950233 0.13872659 0.05457047]]
[[0.00813404 0.41798022 0.29108638 0.08950233 0.13872659 0.05457047]]
[[0.00813404 0.41798013 0.2910864  0.08950236 0.13872664 0.05457049]]


In [22]:
print(get_label_time(predicted_list, 0.30),"\n")

[('disgust', '0:03'), ('disgust', '0:06'), ('disgust', '0:12'), ('disgust', '0:15'), ('disgust', '0:18'), ('disgust', '0:21')] 



In [23]:
for video_name in file_name:
  print(video_name)
  extracted_img = slice_video(video_name)
  extract_face(extracted_img)
  duplicate(extracted_img)
  img_list = transform_img(extracted_img)

  predicted_list = []
  for i in range(0, len(img_list),6):
    input = img_list[i:i+6]
    yhat = trainedModel.predict(input) # , verbose = 1) #Use verbose to get progress bar
    predicted_list.append((yhat.max(), np.argmax(yhat, axis=1)))
    print(yhat,)

  print(get_label_time(predicted_list),"\n")

1meterhappy.mp4
[[0.00894982 0.3735591  0.2917214  0.09683622 0.16858701 0.06034643]]
[[0.00813404 0.41798022 0.29108638 0.08950235 0.13872659 0.05457047]]
[[0.00813404 0.41798022 0.29108638 0.08950233 0.13872659 0.05457047]]
[[0.00813405 0.4179783  0.29108658 0.08950264 0.13872771 0.05457076]]
[[0.00813407 0.41797802 0.2910864  0.08950273 0.13872795 0.05457076]]
[[0.00813488 0.4178966  0.2910919  0.08951686 0.13877797 0.05458185]]
[[0.00813415 0.41795948 0.29108867 0.08950562 0.13873863 0.05457347]]
[[0.00813404 0.41798016 0.29108635 0.08950234 0.13872659 0.05457048]]
[[0.00813405 0.41797954 0.29108644 0.08950242 0.13872692 0.05457057]]
[[0.00813405 0.41797975 0.2910864  0.08950242 0.13872686 0.05457054]]
[('disgust', '0:03'), ('disgust', '0:06'), ('disgust', '0:09'), ('disgust', '0:12'), ('disgust', '0:15'), ('disgust', '0:18'), ('disgust', '0:21'), ('disgust', '0:24'), ('disgust', '0:27')] 

halfmetersad.mp4
[[0.00837561 0.3332132  0.29043067 0.10336053 0.19878654 0.06583356]]
[[0.0